# GBDT - LightGBM

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
import numpy as np
import pandas as pd
import lightgbm
import pickle
import json
import functools
from os import listdir
from os.path import isfile, join
from nilib import *

In [3]:
def train_gradient_boosting_baseline(train_file, valid_file, test_file, output_model_file, 
                                     drop_cols=None, random_forest = False):
    
    exp = pd.DataFrame(columns=['num_trees', 'learning_rate', 'num_leaves', 'best_round', 'metric', 'filename'])
    
    # load train/valid/test
    train, valid, test = load_atk_train_valid_test(train_file, valid_file, test_file)
    
    assert "instance_id" not in train.columns.values, "Wrong training set file for GBDT"

    # dropping some of the columns
    if drop_cols is not None:
        print ("Dropping columns:", drop_cols)
        train.drop(columns=drop_cols, inplace=True)
        valid.drop(columns=drop_cols, inplace=True)
        test.drop(columns=drop_cols, inplace=True)
    

    for num_trees in [100]:#100
        for learning_rate in [0.05]: #[0.01, 0.05]:
            for num_leaves in [2**8]: #[16, 24]:
                # datasets
                lgbm_train = lightgbm.Dataset(data=train.iloc[:,:-1], 
                                              label=train.iloc[:,-1])

                lgbm_valid = lightgbm.Dataset(data=valid.iloc[:,:-1], 
                                              label=valid.iloc[:,-1])

                # run train
                lgbm_params = { 'learning_rate': learning_rate, 
                                'num_leaves': num_leaves,
                                'max_depth': 8,
                                'objective': 'regression' #'binary'
                              } 
                loss_name = 'l2'#'binary_logloss' 
                if random_forest:
                    lgbm_params['boosting'] = 'rf'
                    lgbm_params['bagging_fraction'] = 0.8
                    lgbm_params['feature_fraction' ] = 0.8
                    lgbm_params['bagging_freq'] = 1

                lgbm_info = {}
                lgbm_model = lightgbm.train(lgbm_params, lgbm_train, 
                                            num_boost_round = num_trees,
#                                             fobj            = optimize_log_loss, 
#                                             feval           = avg_log_loss,
                                            evals_result    = lgbm_info,
                                            valid_sets      = [lgbm_train, lgbm_valid], 
                                            valid_names     = ['train', 'valid'],
                                            verbose_eval    = 10)
                
                best_valid_iter = np.argmin(lgbm_info['valid'][loss_name])
                
                model_file_name = "{:s}_T{:d}_S{:04d}_L{:d}_R{:d}.model".format(output_model_file,
                                                                            num_trees,
                                                                            int(learning_rate*1000),
                                                                            num_leaves,
                                                                            best_valid_iter + 1
                                                                           )
                
                # update experimental results
                exp = exp.append({'num_trees': num_trees, 
                                  'learning_rate':learning_rate,
                                  'num_leaves':num_leaves, 
                                  'best_round':best_valid_iter+1, 
                                  'metric':lgbm_info['valid'][loss_name][best_valid_iter],
                                  'filename':model_file_name},
                                   ignore_index=True)
                
        
                lgbm_model.save_model(model_file_name)
                print("Model saved to", model_file_name)
    
    return exp

# WINE Dataset

In [5]:
DATASET_NAME="wine"
DATASET_DIR="../data/{}".format(DATASET_NAME)
ATK_DIR=DATASET_DIR + "/attacks"
MODELS_DIR="../out/models/{}".format(DATASET_NAME)
TRAINING_FILENAME=DATASET_DIR + "/" + "train.csv.bz2"
VALIDATION_FILENAME=DATASET_DIR + "/" + "valid.csv.bz2"
TEST_FILENAME=DATASET_DIR + "/" + "test.csv.bz2"
MODEL_FILENAME=MODELS_DIR + "/std-gbdt_{}".format(DATASET_NAME)
RED_MODEL_FILENAME=MODELS_DIR + "/red-gbdt_{}".format(DATASET_NAME)
RF_MODEL_FILENAME=MODELS_DIR + "/rf-gbdt_{}".format(DATASET_NAME)

In [6]:
# FULL GDBT

experiments = train_gradient_boosting_baseline(TRAINING_FILENAME, 
                                               VALIDATION_FILENAME,
                                               TEST_FILENAME,
                                               MODEL_FILENAME
                                              )  

experiments.to_csv(MODEL_FILENAME + ".csv", index=False)

print(experiments)
print ('best model is:', experiments.sort_values('metric').iloc[0]['filename'] )

Pre-processing original files...
Loading: ../data/wine/train.csv.bz2
Loading: ../data/wine/valid.csv.bz2
Loading: ../data/wine/test.csv.bz2
Train/Valid/Test sizes: (4547, 13) (650, 13) (1300, 13)
Train/Valid/Test split: 0.70 0.10 0.20
CatFX: []
Train/Valid/Test sizes: (3898, 13) (1299, 13) (1300, 13)
Train/Valid/Test split: 0.60 0.20 0.20
Saving processed files *.atks.bz2
[10]	train's l2: 0.658938	valid's l2: 0.73916
[20]	train's l2: 0.527671	valid's l2: 0.660676
[30]	train's l2: 0.451371	valid's l2: 0.624959
[40]	train's l2: 0.407846	valid's l2: 0.609611
[50]	train's l2: 0.371556	valid's l2: 0.598689
[60]	train's l2: 0.34594	valid's l2: 0.593508
[70]	train's l2: 0.331191	valid's l2: 0.58887
[80]	train's l2: 0.317604	valid's l2: 0.585445
[90]	train's l2: 0.306344	valid's l2: 0.583606
[100]	train's l2: 0.292753	valid's l2: 0.58101
Model saved to ../out/models/wine/std-gbdt_wine_T100_S0050_L256_R100.model
  num_trees  learning_rate num_leaves best_round   metric  \
0       100           

In [ ]:
# REDUCED GDBT

experiments = train_gradient_boosting_baseline(TRAINING_FILENAME, 
                                               VALIDATION_FILENAME,
                                               TEST_FILENAME,
                                               RED_MODEL_FILENAME,
                                               drop_cols=["alcohol", "residual_sugar", "volatile_acidity"]
                                              )  

experiments.to_csv(RED_MODEL_FILENAME + ".csv", index=False)

print(experiments)
print ('best model is:', experiments.sort_values('metric').iloc[0]['filename'] )

In [7]:
# RANDOM FOREST

experiments = train_gradient_boosting_baseline(TRAINING_FILENAME, 
                                               VALIDATION_FILENAME,
                                               TEST_FILENAME,
                                               RF_MODEL_FILENAME,
                                               random_forest=True
                                              )  

experiments.to_csv(RF_MODEL_FILENAME + ".csv", index=False)

print(experiments)
print ('best model is:', experiments.sort_values('metric').iloc[0]['filename'] )

Pre-processing original files...
Loading: ../data/wine/train.csv.bz2
Loading: ../data/wine/valid.csv.bz2
Loading: ../data/wine/test.csv.bz2
Train/Valid/Test sizes: (4547, 13) (650, 13) (1300, 13)
Train/Valid/Test split: 0.70 0.10 0.20
CatFX: []
Train/Valid/Test sizes: (3898, 13) (1299, 13) (1300, 13)
Train/Valid/Test split: 0.60 0.20 0.20
Saving processed files *.atks.bz2
[10]	train's l2: 0.520077	valid's l2: 0.62858
[20]	train's l2: 0.512319	valid's l2: 0.626062
[30]	train's l2: 0.507472	valid's l2: 0.621204
[40]	train's l2: 0.506059	valid's l2: 0.62197
[50]	train's l2: 0.503661	valid's l2: 0.621407
[60]	train's l2: 0.50284	valid's l2: 0.620475
[70]	train's l2: 0.503065	valid's l2: 0.620394
[80]	train's l2: 0.503063	valid's l2: 0.620651
[90]	train's l2: 0.502976	valid's l2: 0.620229
[100]	train's l2: 0.502296	valid's l2: 0.620133
Model saved to ../out/models/wine/rf-gbdt_wine_T100_S0050_L256_R68.model
  num_trees  learning_rate num_leaves best_round    metric  \
0       100           

# CENSUS Dataset

In [ ]:
DATASET_NAME="census"
DATASET_DIR="../data/{}".format(DATASET_NAME)
ATK_DIR=DATASET_DIR + "/attacks"
MODELS_DIR="../out/models/{}".format(DATASET_NAME)
TRAINING_FILENAME=DATASET_DIR + "/" + "train.csv.bz2"
VALIDATION_FILENAME=DATASET_DIR + "/" + "valid.csv.bz2"
TEST_FILENAME=DATASET_DIR + "/" + "test.csv.bz2"
MODEL_FILENAME=MODELS_DIR + "/std-gbdt_{}".format(DATASET_NAME)
RED_MODEL_FILENAME=MODELS_DIR + "/red-gbdt_{}".format(DATASET_NAME)
RF_MODEL_FILENAME=MODELS_DIR + "/rf-gbdt_{}".format(DATASET_NAME)

In [ ]:
# FULL GDBT

experiments = train_gradient_boosting_baseline(TRAINING_FILENAME, 
                                               VALIDATION_FILENAME,
                                               TEST_FILENAME,
                                               MODEL_FILENAME
                                              )  

experiments.to_csv(MODEL_FILENAME + ".csv", index=False)

print(experiments)
print ('best model is:', experiments.sort_values('metric').iloc[0]['filename'] )

In [ ]:
# REDUCED GDBT

experiments = train_gradient_boosting_baseline(TRAINING_FILENAME, 
                                               VALIDATION_FILENAME,
                                               TEST_FILENAME,
                                               RED_MODEL_FILENAME,
                                               drop_cols=['workclass','marital_status',
                                                          'occupation', 'education_num',
                                                          'hours_per_week','capital_gain' ]
                                              )

experiments.to_csv(RED_MODEL_FILENAME + ".csv", index=False)

print(experiments)
print ('best model is:', experiments.sort_values('metric').iloc[0]['filename'] )

In [ ]:
# RANDOM FOREST

experiments = train_gradient_boosting_baseline(TRAINING_FILENAME, 
                                               VALIDATION_FILENAME,
                                               TEST_FILENAME,
                                               RF_MODEL_FILENAME,
                                               random_forest=True
                                              )  

experiments.to_csv(RF_MODEL_FILENAME + ".csv", index=False)

print(experiments)
print ('best model is:', experiments.sort_values('metric').iloc[0]['filename'] )

# SPAM Dataset

In [ ]:
DATASET_NAME="spam"
DATASET_DIR="../data/{}".format(DATASET_NAME)
ATK_DIR=DATASET_DIR + "/attacks"
MODELS_DIR="../out/models/{}".format(DATASET_NAME)
TRAINING_FILENAME=DATASET_DIR + "/" + "train.csv.bz2"
VALIDATION_FILENAME=DATASET_DIR + "/" + "valid.csv.bz2"
TEST_FILENAME=DATASET_DIR + "/" + "test.csv.bz2"
MODEL_FILENAME=MODELS_DIR + "/std-gbdt_{}".format(DATASET_NAME)
RED_MODEL_FILENAME=MODELS_DIR + "/red-gbdt_{}".format(DATASET_NAME)
RF_MODEL_FILENAME=MODELS_DIR + "/rf-gbdt_{}".format(DATASET_NAME)

In [ ]:
# FULL GDBT

experiments = train_gradient_boosting_baseline(TRAINING_FILENAME, 
                                               VALIDATION_FILENAME,
                                               TEST_FILENAME,
                                               MODEL_FILENAME
                                              )  

experiments.to_csv(MODEL_FILENAME + ".csv", index=False)

print(experiments)
print ('best model is:', experiments.sort_values('metric').iloc[0]['filename'] )

In [ ]:
# REDUCED GDBT

experiments = train_gradient_boosting_baseline(TRAINING_FILENAME, 
                                               VALIDATION_FILENAME,
                                               TEST_FILENAME,
                                               RED_MODEL_FILENAME,
                                               drop_cols=['char_freq_!', 'word_freq_remove',
                                                          'char_freq_$', 'capital_run_length_average',
                                                          'capital_run_length_total' ]
                                              )

experiments.to_csv(RED_MODEL_FILENAME + ".csv", index=False)

print(experiments)
print ('best model is:', experiments.sort_values('metric').iloc[0]['filename'] )

In [ ]:
# RANDOM FOREST

experiments = train_gradient_boosting_baseline(TRAINING_FILENAME, 
                                               VALIDATION_FILENAME,
                                               TEST_FILENAME,
                                               RF_MODEL_FILENAME,
                                               random_forest=True
                                              )  

experiments.to_csv(RF_MODEL_FILENAME + ".csv", index=False)

print(experiments)
print ('best model is:', experiments.sort_values('metric').iloc[0]['filename'] )

# debug


In [ ]:
###------------_####
def print_fx_imp(model, colnames):
    fx_uses = model.feature_importance(importance_type='split')
    fx_gain = model.feature_importance(importance_type='gain')

    for i,f in enumerate(np.argsort(fx_gain)[::-1]):
        print ("{:2d} {:20s} {:.3f} {:4d}".format(i, colnames[f], fx_gain[f], fx_uses[f]))

        




In [ ]:
df = pd.read_csv("../data/wine/raw/train.csv.bz2")

In [ ]:
print(" -- GDBT --")    
gbdt = lightgbm.Booster(model_file="../out/models/wine/std-gbdt_wine_T100_S0050_L256_R97.model")
print(gbdt.num_trees())
print_fx_imp(gbdt, df.columns)

print(" -- Reduced GDBT --")    
redf = lightgbm.Booster(model_file="../out/models/wine/red-gbdt_wine_T100_S0050_L256_R100.model")
print(redf.num_trees())
#print_fx_imp(redf,train.drop(columns=['char_freq_!', 'word_freq_remove',
#                                                          'char_freq_$', 'capital_run_length_average',
#                                                         'capital_run_length_total', 'word_freq_hp' ]).columns
#            )


# print(" -- Adv. Boosting --")    
# advb = lightgbm.Booster(model_file="../out/models/census/adv-boosting_census_B30_T100_S0050_L24_R100.model")
# print(advb.num_trees())
# print_fx_imp(advb, TRAIN.columns)

# CREDIT Dataset

In [ ]:
DATASET_NAME="credit"
DATASET_DIR="../data/{}".format(DATASET_NAME)
ATK_DIR=DATASET_DIR + "/attacks"
MODELS_DIR="../out/models/{}".format(DATASET_NAME)
TRAINING_FILENAME=DATASET_DIR + "/" + "train.csv.bz2"
VALIDATION_FILENAME=DATASET_DIR + "/" + "valid.csv.bz2"
TEST_FILENAME=DATASET_DIR + "/" + "test.csv.bz2"
MODEL_FILENAME=MODELS_DIR + "/std-gbdt_{}".format(DATASET_NAME)
RED_MODEL_FILENAME=MODELS_DIR + "/red-gbdt_{}".format(DATASET_NAME)
RF_MODEL_FILENAME=MODELS_DIR + "/rf-gbdt_{}".format(DATASET_NAME)

In [ ]:
# FULL GDBT

experiments = train_gradient_boosting_baseline(TRAINING_FILENAME, 
                                               VALIDATION_FILENAME,
                                               TEST_FILENAME,
                                               MODEL_FILENAME
                                              )  

experiments.to_csv(MODEL_FILENAME + ".csv", index=False)

print(experiments)
print ('best model is:', experiments.sort_values('metric').iloc[0]['filename'] )

In [ ]:
# REDUCED GDBT

experiments = train_gradient_boosting_baseline(TRAINING_FILENAME, 
                                               VALIDATION_FILENAME,
                                               TEST_FILENAME,
                                               RED_MODEL_FILENAME,
                                               drop_cols=['PAY_0', 'BILL_AMT1', 'PAY_2', 'LIMIT_BAL'] # SET TO THE LIST OF ATTACKED FEATURES
                                              )

experiments.to_csv(RED_MODEL_FILENAME + ".csv", index=False)

print(experiments)
print ('best model is:', experiments.sort_values('metric').iloc[0]['filename'] )

In [ ]:
# RANDOM FOREST

experiments = train_gradient_boosting_baseline(TRAINING_FILENAME, 
                                               VALIDATION_FILENAME,
                                               TEST_FILENAME,
                                               RF_MODEL_FILENAME,
                                               random_forest=True
                                              )  

experiments.to_csv(RF_MODEL_FILENAME + ".csv", index=False)

print(experiments)
print ('best model is:', experiments.sort_values('metric').iloc[0]['filename'] )

# WEBSITES Dataset

In [ ]:
DATASET_NAME="websites"
DATASET_DIR="../data/{}".format(DATASET_NAME)
ATK_DIR=DATASET_DIR + "/attacks"
MODELS_DIR="../out/models/{}".format(DATASET_NAME)
TRAINING_FILENAME=DATASET_DIR + "/" + "train.csv.bz2"
VALIDATION_FILENAME=DATASET_DIR + "/" + "valid.csv.bz2"
TEST_FILENAME=DATASET_DIR + "/" + "test.csv.bz2"
MODEL_FILENAME=MODELS_DIR + "/std-gbdt_{}".format(DATASET_NAME)
RED_MODEL_FILENAME=MODELS_DIR + "/red-gbdt_{}".format(DATASET_NAME)
RF_MODEL_FILENAME=MODELS_DIR + "/rf-gbdt_{}".format(DATASET_NAME)

In [ ]:
# FULL GDBT

experiments = train_gradient_boosting_baseline(TRAINING_FILENAME, 
                                               VALIDATION_FILENAME,
                                               TEST_FILENAME,
                                               MODEL_FILENAME
                                              )  

experiments.to_csv(MODEL_FILENAME + ".csv", index=False)

print(experiments)
print ('best model is:', experiments.sort_values('metric').iloc[0]['filename'] )

In [ ]:
# REDUCED GDBT

experiments = train_gradient_boosting_baseline(TRAINING_FILENAME, 
                                               VALIDATION_FILENAME,
                                               TEST_FILENAME,
                                               RED_MODEL_FILENAME,
                                               drop_cols=[] # SET TO THE LIST OF ATTACKED FEATURES
                                              )

experiments.to_csv(RED_MODEL_FILENAME + ".csv", index=False)

print(experiments)
print ('best model is:', experiments.sort_values('metric').iloc[0]['filename'] )

In [ ]:
# RANDOM FOREST

experiments = train_gradient_boosting_baseline(TRAINING_FILENAME, 
                                               VALIDATION_FILENAME,
                                               TEST_FILENAME,
                                               RF_MODEL_FILENAME,
                                               random_forest=True
                                              )  

experiments.to_csv(RF_MODEL_FILENAME + ".csv", index=False)

print(experiments)
print ('best model is:', experiments.sort_values('metric').iloc[0]['filename'] )

# Financial Distress 

In [ ]:
DATASET_NAME="financial"
DATASET_DIR="../data/{}".format(DATASET_NAME)
ATK_DIR=DATASET_DIR + "/attacks"
MODELS_DIR="../out/models/{}".format(DATASET_NAME)
TRAINING_FILENAME=DATASET_DIR + "/" + "train.csv.bz2"
VALIDATION_FILENAME=DATASET_DIR + "/" + "valid.csv.bz2"
TEST_FILENAME=DATASET_DIR + "/" + "test.csv.bz2"
MODEL_FILENAME=MODELS_DIR + "/std-gbdt_{}".format(DATASET_NAME)
RED_MODEL_FILENAME=MODELS_DIR + "/red-gbdt_{}".format(DATASET_NAME)
RF_MODEL_FILENAME=MODELS_DIR + "/rf-gbdt_{}".format(DATASET_NAME)

In [ ]:
# FULL GDBT

experiments = train_gradient_boosting_baseline(TRAINING_FILENAME, 
                                               VALIDATION_FILENAME,
                                               TEST_FILENAME,
                                               MODEL_FILENAME
                                              )  

experiments.to_csv(MODEL_FILENAME + ".csv", index=False)

print(experiments)
print ('best model is:', experiments.sort_values('metric').iloc[0]['filename'] )

In [ ]:
# REDUCED GDBT

experiments = train_gradient_boosting_baseline(TRAINING_FILENAME, 
                                               VALIDATION_FILENAME,
                                               TEST_FILENAME,
                                               RED_MODEL_FILENAME,
                                               drop_cols=['x36','x37','x81','x47','x40','x3','x16','x52'] # SET TO THE LIST OF ATTACKED FEATURES
                                              )

experiments.to_csv(RED_MODEL_FILENAME + ".csv", index=False)

print(experiments)
print ('best model is:', experiments.sort_values('metric').iloc[0]['filename'] )

In [ ]:
# RANDOM FOREST

experiments = train_gradient_boosting_baseline(TRAINING_FILENAME, 
                                               VALIDATION_FILENAME,
                                               TEST_FILENAME,
                                               RF_MODEL_FILENAME,
                                               random_forest=True
                                              )  

experiments.to_csv(RF_MODEL_FILENAME + ".csv", index=False)

print(experiments)
print ('best model is:', experiments.sort_values('metric').iloc[0]['filename'] )

In [ ]:
###------------_####
def print_fx_imp(model, colnames):
    fx_uses = model.feature_importance(importance_type='split')
    fx_gain = model.feature_importance(importance_type='gain')

    for i,f in enumerate(np.argsort(fx_gain)[::-1]):
        print ("{:2d} {:20s} {:.3f} {:4d}".format(i, colnames[f], fx_gain[f], fx_uses[f]))

        




In [ ]:
df = pd.read_csv("../data/financial/raw/dataset.csv")

In [ ]:
print(" -- GDBT --")    
gbdt = lightgbm.Booster(model_file="../out/models/financial/std-gbdt_financial_T100_S0050_L256_R37.model")
print(gbdt.num_trees())
print_fx_imp(gbdt, df.columns)

#print(" -- Reduced GDBT --")    
#redf = lightgbm.Booster(model_file="../out/models/wine/red-gbdt_wine_T100_S0050_L256_R100.model")
#print(redf.num_trees())
#print_fx_imp(redf,train.drop(columns=['char_freq_!', 'word_freq_remove',
#                                                          'char_freq_$', 'capital_run_length_average',
#                                                         'capital_run_length_total', 'word_freq_hp' ]).columns
#            )


# print(" -- Adv. Boosting --")    
# advb = lightgbm.Booster(model_file="../out/models/census/adv-boosting_census_B30_T100_S0050_L24_R100.model")
# print(advb.num_trees())
# print_fx_imp(advb, TRAIN.columns)

# malware_all

In [4]:
DATASET_NAME="malware_all"
DATASET_DIR="../data/{}".format(DATASET_NAME)
ATK_DIR=DATASET_DIR + "/attacks"
MODELS_DIR="../out/models/{}".format(DATASET_NAME)
TRAINING_FILENAME=DATASET_DIR + "/" + "train.csv.bz2"
VALIDATION_FILENAME=DATASET_DIR + "/" + "valid.csv.bz2"
TEST_FILENAME=DATASET_DIR + "/" + "test.csv.bz2"
MODEL_FILENAME=MODELS_DIR + "/std-gbdt_{}".format(DATASET_NAME)
RED_MODEL_FILENAME=MODELS_DIR + "/red-gbdt_{}".format(DATASET_NAME)
RF_MODEL_FILENAME=MODELS_DIR + "/rf-gbdt_{}".format(DATASET_NAME)

In [6]:
# FULL GDBT

experiments = train_gradient_boosting_baseline(TRAINING_FILENAME, 
                                               VALIDATION_FILENAME,
                                               TEST_FILENAME,
                                               MODEL_FILENAME
                                              )  

experiments.to_csv(MODEL_FILENAME + ".csv", index=False)

print(experiments)
print ('best model is:', experiments.sort_values('metric').iloc[0]['filename'] )

Pre-processing original files...
Loading: ../data/malware_all/train.csv.bz2
Loading: ../data/malware_all/valid.csv.bz2
Loading: ../data/malware_all/test.csv.bz2
Train/Valid/Test sizes: (32028, 1001) (10626, 1001) (9516, 1001)
Train/Valid/Test split: 0.61 0.20 0.18
CatFX: []
Train/Valid/Test sizes: (31302, 1001) (10434, 1001) (10434, 1001)
Train/Valid/Test split: 0.60 0.20 0.20
Saving processed files *.atks.bz2
[10]	train's l2: 0.483024	valid's l2: 0.500222
[20]	train's l2: 0.289569	valid's l2: 0.321536
[30]	train's l2: 0.208312	valid's l2: 0.248792
[40]	train's l2: 0.172017	valid's l2: 0.219058
[50]	train's l2: 0.151924	valid's l2: 0.204255
[60]	train's l2: 0.138811	valid's l2: 0.197557
[70]	train's l2: 0.131712	valid's l2: 0.194469
[80]	train's l2: 0.124403	valid's l2: 0.189363
[90]	train's l2: 0.118814	valid's l2: 0.18638
[100]	train's l2: 0.114558	valid's l2: 0.186027
Model saved to ../out/models/malware_all/std-gbdt_malware_all_T100_S0050_L256_R100.model
  num_trees  learning_rate 

In [ ]:
# REDUCED GDBT

experiments = train_gradient_boosting_baseline(TRAINING_FILENAME, 
                                               VALIDATION_FILENAME,
                                               TEST_FILENAME,
                                               RED_MODEL_FILENAME,
                                               drop_cols=[] # SET TO THE LIST OF ATTACKED FEATURES
                                              )

experiments.to_csv(RED_MODEL_FILENAME + ".csv", index=False)

print(experiments)
print ('best model is:', experiments.sort_values('metric').iloc[0]['filename'] )

In [7]:
# RANDOM FOREST

experiments = train_gradient_boosting_baseline(TRAINING_FILENAME, 
                                               VALIDATION_FILENAME,
                                               TEST_FILENAME,
                                               RF_MODEL_FILENAME,
                                               random_forest=True
                                              )  

experiments.to_csv(RF_MODEL_FILENAME + ".csv", index=False)

print(experiments)
print ('best model is:', experiments.sort_values('metric').iloc[0]['filename'] )

Pre-processing original files...
Loading: ../data/malware_all/train.csv.bz2
Loading: ../data/malware_all/valid.csv.bz2
Loading: ../data/malware_all/test.csv.bz2
Train/Valid/Test sizes: (32028, 1001) (10626, 1001) (9516, 1001)
Train/Valid/Test split: 0.61 0.20 0.18
CatFX: []
Train/Valid/Test sizes: (31302, 1001) (10434, 1001) (10434, 1001)
Train/Valid/Test split: 0.60 0.20 0.20
Saving processed files *.atks.bz2
[10]	train's l2: 0.198602	valid's l2: 0.239825
[20]	train's l2: 0.195415	valid's l2: 0.232913
[30]	train's l2: 0.193919	valid's l2: 0.230579
[40]	train's l2: 0.194236	valid's l2: 0.231758
[50]	train's l2: 0.193531	valid's l2: 0.229268
[60]	train's l2: 0.193172	valid's l2: 0.228287
[80]	train's l2: 0.192708	valid's l2: 0.22864
[90]	train's l2: 0.192576	valid's l2: 0.228547
[100]	train's l2: 0.192432	valid's l2: 0.228207
Model saved to ../out/models/malware_all/rf-gbdt_malware_all_T100_S0050_L256_R97.model
  num_trees  learning_rate num_leaves best_round    metric  \
0       100   